Часть материала украдена из курса "Глубинное обучение" ФКН ВШЭ https://www.hse.ru/ba/ami/courses/205504078.html, за что им большое спасибо.

Актуальная версия этого ноутбука обретается по адресу
https://github.com/nadiinchi/dl_labs/blob/master/lab_pytorch.ipynb

# Устанавливаем pytorch

## Linux/OSX


На оффсайте http://pytorch.org/ надо выбрать подходящую конфигурацию и скачать.

Версию python можно узнать в терминале:
```
python --version
```


## Windows without GPU

Проще всего поставить при помощи конды:
```
conda install -c peterjc123 pytorch
```

## Windows with GPU

Смотрите https://github.com/peterjc123/pytorch-scripts

![img](https://s1.postimg.org/6fl45xnvnj/pytorch-logo-dark.png)

In [1]:
import torch
import numpy as np

In [2]:
# numpy world

x = np.arange(16).reshape(4, 4)

print("X :\n %s" % x)
print("add 5 :\n%s" % (x + 5))
print("X*X^T  :\n", np.dot(x, x.T))
print("mean over cols :\n%s" % (x.mean(axis=-1)))
print("cumsum of cols :\n%s" % (np.cumsum(x, axis=0)))

X :
 [[ 0  1  2  3]
 [ 4  5  6  7]
 [ 8  9 10 11]
 [12 13 14 15]]
add 5 :
[[ 5  6  7  8]
 [ 9 10 11 12]
 [13 14 15 16]
 [17 18 19 20]]
X*X^T  :
 [[ 14  38  62  86]
 [ 38 126 214 302]
 [ 62 214 366 518]
 [ 86 302 518 734]]
mean over cols :
[  1.5   5.5   9.5  13.5]
cumsum of cols :
[[ 0  1  2  3]
 [ 4  6  8 10]
 [12 15 18 21]
 [24 28 32 36]]


In [3]:
# pytorch world

x = np.arange(16).reshape(4, 4)

y = torch.from_numpy(x)
print('Type of y:', type(y))
y = torch.from_numpy(x).type(torch.FloatTensor)
print('Type of y', type(y))

Type of y: <class 'torch.IntTensor'>
Type of y <class 'torch.FloatTensor'>


In [4]:
x = torch.arange(0,16).view(4,4)
print('Type of x:', type(x))

print("X :\n%s" % x)
print("add 5 :\n%s" % (x + 5))
print("X*X^T  :\n", torch.matmul(x, x.transpose(1, 0)))
print("mean over cols :\n", torch.mean(x, dim=-1))
print("cumsum of cols :\n", torch.cumsum(x, dim=0))

Type of x: <class 'torch.FloatTensor'>
X :

  0   1   2   3
  4   5   6   7
  8   9  10  11
 12  13  14  15
[torch.FloatTensor of size 4x4]

add 5 :

  5   6   7   8
  9  10  11  12
 13  14  15  16
 17  18  19  20
[torch.FloatTensor of size 4x4]

X*X^T  :
 
  14   38   62   86
  38  126  214  302
  62  214  366  518
  86  302  518  734
[torch.FloatTensor of size 4x4]

mean over cols :
 
  1.5000
  5.5000
  9.5000
 13.5000
[torch.FloatTensor of size 4]

cumsum of cols :
 
  0   1   2   3
  4   6   8  10
 12  15  18  21
 24  28  32  36
[torch.FloatTensor of size 4x4]



## NumPy vs Pytorch

Numpy и Pytorch не требуют описания статического графа вычислений. 

Можно отлаживаться с помощью pdb или просто print.

API несколько различается:

```
x.reshape([1,2,8]) -> x.view(1,2,8)
x.sum(axis=-1) -> x.sum(dim=-1)
x.astype('int64') -> x.type(torch.LongTensor)
```


Легко конвертировать между собой:

```
torch.from_numpy(npx) -- вернет Tensor
tt.numpy() -- вернет Numpy Array
```


Если что:
- смотрите документацию
- гуглите (Stackoverflow/tutorials)

In [5]:
x = torch.linspace(0, 2 * np.pi, 16)

# Mini-task: compute a vector of sin^2(x) + cos^2(x)
out = torch.sin(x)**2 + torch.cos(x)**2

print(out.numpy())

[ 1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.]


In [6]:
out


 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
[torch.FloatTensor of size 16]

## In-place operations

Когда работаем с большими массивами, память надо экономить.
Некоторые операции происходят с созданием нового объекта – результата вычислений,
некоторые изменяют данный объект (in-place операции).
В pytorch обычно эти операции различаются добавлением подчеркивания:
```
x.exp()   # not-in-place operation
x.exp_()  # in-place operation
```

In [7]:
x = torch.arange(4)
print('Not-in-place:')
print('\tx.exp():\t\t', x.exp().numpy())
print('\tx:\t\t\t', x.numpy())
print('In-place:')
print('\tx.exp_():\t\t', x.exp_().numpy())
print('\tx after x.exp_():\t', x.numpy())

Not-in-place:
	x.exp():		 [  1.           2.71828175   7.38905621  20.08553696]
	x:			 [ 0.  1.  2.  3.]
In-place:
	x.exp_():		 [  1.           2.71828175   7.38905621  20.08553696]
	x after x.exp_():	 [  1.           2.71828175   7.38905621  20.08553696]


In [8]:
x = torch.arange(0, 4).view(2, 2)
y = torch.arange(4, 8).view(2, 2)
z = torch.arange(8, 12).view(2, 2)

# Not-in-place:
u = x + 2 * y - z    # 3 array allocations?
print(u.numpy())

# In-place
u = y.clone()        # 1 array allocation
u.mul_(2)
u.add_(x)
u.sub_(z)
print(u.numpy())

[[ 0.  2.]
 [ 4.  6.]]
[[ 0.  2.]
 [ 4.  6.]]


## Broadcasting на pytorch (аналогично numpy)

In [9]:
a = torch.Tensor([1, 1, 1, 2]).view(4, 1)
b = torch.Tensor([1, 0, 1, 0]).view(1, 4)
c = torch.randn(16).view(4, 4)
print('a:', a)
print('b:', b)
print('a + b:', a + b)
print('c:', c)
print('b + c:', b + c)

a: 
 1
 1
 1
 2
[torch.FloatTensor of size 4x1]

b: 
 1  0  1  0
[torch.FloatTensor of size 1x4]

a + b: 
 2  1  2  1
 2  1  2  1
 2  1  2  1
 3  2  3  2
[torch.FloatTensor of size 4x4]

c: 
 0.1850 -0.1429 -1.0659  0.3823
 0.9214 -1.2254 -0.0475 -0.1747
-0.8821 -0.6159  1.1399 -2.3033
 2.1796  0.0658  3.3187 -2.2893
[torch.FloatTensor of size 4x4]

b + c: 
 1.1850 -0.1429 -0.0659  0.3823
 1.9214 -1.2254  0.9525 -0.1747
 0.1179 -0.6159  2.1399 -2.3033
 3.1796  0.0658  4.3187 -2.2893
[torch.FloatTensor of size 4x4]



Более подробную информацию можно найти на http://pytorch.org/docs/master/notes/broadcasting.html

## Работа с тензорами

Дано 100 объектов, каждый из которых описывается 10-мерным вектором, и 5 точек, каждая из которых также задается 10-мерным вектором. Объекты лежат в матрице X, точки – в матрице Y.

Надо для каждого объекта из X найти индекс ближайшей точки из Y только с помощью операций над тензорами
(нельзя использовать циклы, list comprehensions, рекурсию, etc,
потому что решение с ними будет работать в несколько раз или на несколько порядков медленнее).

In [10]:
X = torch.randn(100, 10)
Y = torch.randn(5, 10)

Решение с семинара:

In [11]:
((X.view(100, 1, 10) - Y.view(1, 5, 10)) ** 2).sum(dim=-1).min(dim=-1)[1].numpy()

array([0, 1, 2, 2, 2, 2, 2, 2, 2, 2, 4, 4, 4, 2, 2, 1, 2, 2, 2, 2, 3, 2, 0,
       2, 2, 2, 0, 0, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 1, 2, 2, 2, 4, 2, 2,
       2, 1, 0, 2, 2, 2, 2, 2, 2, 2, 2, 4, 4, 2, 0, 4, 4, 2, 1, 3, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 4, 0, 2, 2, 4, 2, 2, 2, 0, 2, 2, 2, 2, 3, 2,
       2, 2, 2, 2, 3, 2, 2, 2], dtype=int64)

Это решение плохо тем, что в качестве промежуточного результата вычилений в нем присутствует трехмерный тензор,
который занимает $O(NMD)$ памяти, где N – число объектов, M – число точек, D – размерность пространства.

## Внимание, задача!
Утверждается, что есть другое решение с такой же скоростью работы,
но использующее $O(NM)$ памяти для результатов промежуточных вычислений.
Предлагается найти его.

Подсказка: найти матрицу попарных скалярных произведений между объектами
и точками можно с помощью одного матричного умножения.

In [12]:
# your code here
#c**2 = a**2 + b**2 -2abcos(a), 2abcos(a) - scalar multiplication of a&b
XY_scalar = torch.matmul(X, Y.transpose(0, 1))
X_2 = (X**2).sum(dim=1)
Y_2 = (Y**2).sum(dim=1)

In [13]:
(Y_2.view(1, 5)+X_2.view(100, 1) - XY_scalar).min(dim=1)[1].numpy()

array([2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 4, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2], dtype=int64)

## CUDA
`x.cuda()` копирует тензор на GPU и возвращает объект, соответствующий этому скопированному тензору.
Можно явно указать номер GPU, на который нужно скопировать тензор: `x.cuda(gpu_id)`.
Если тензор уже лежал на нужном GPU, то возвращается сам тензор, копирования не производится.
Аналогично работает `x.cpu()`. 

Операции можно осуществлять только над тензорами, лежащими на одном устройстве.
Нарушение этого правила приводит к ошибке.
Результат операции находится на том же устройстве, что и операнды.

# Tensor vs Variable

http://pytorch.org/docs/master/autograd.html#variable

`Variable` – обертка над Tensor для использования в вычислительных графах. Позволяет вычислять градиенты автоматически.

Tensor и Variable конвертируются друг в друга:
```
tensor to variable: Variable(x)
variable to tensor: x.data
```

Нельзя смешивать Tensor и Variable в одной операции.

Некоторые операции могут работать только с тензорами, некоторые только с переменными (torch.nn.functional.whatever).

In [14]:
from torch.autograd import Variable

sequence = torch.randn(1, 8, 10)
filters = torch.randn(2, 8, 3)
a = torch.randn(5)
b = torch.randn(5)

In [15]:
# works:
print('sum of Variables:')
print(Variable(a) + Variable(b))

sum of Variables:
Variable containing:
-0.9251
 0.4932
 0.1173
 1.4256
 0.4002
[torch.FloatTensor of size 5]



In [16]:
# will not work:
print('sum of Variable and Tensor:')
print(Variable(a) + b)

sum of Variable and Tensor:


RuntimeError: add() received an invalid combination of arguments - got (torch.FloatTensor), but expected one of:
 * (float other, float alpha)
 * (Variable other, float alpha)


In [ ]:
# works:
print('conv1d over Variables:')
print(torch.nn.functional.conv1d(Variable(sequence), Variable(filters)))

In [ ]:
# will not work
print("conv1d (tensors):")
print(torch.nn.functional.conv1d(sequence, filters))

# Automatic gradients

Автоматическое вычисление градиентов:

1. Создать переменную: `a = Variable(..., requires_grad=True)`

2. Определить какую-нибудь дифференцируемую _скалярную_ функцию `loss = whatever(a)`

3. Запросить обратный проход `loss.backward()`

4. Градиенты будут доступны в `a.grads`


Есть два важных отличия Pytorch от Theano/TF:

1. Функцию ошибки можно изменять динамически, например на каждом минибатче.

2. После вычисления `.backward()` градиенты сохраняются в `.grad` каждой задействованной переменной, при повторных вызовах градиенты суммируются. Это позволяет использовать несколько функций ошибок или виртуально увеличивать batch_size. Поэтому после каждого шага оптимизатора градиенты стоит обнулять.

### Простой пример использования

In [ ]:
x_tensor = torch.randn(4)
y_tensor = torch.randn(4)
x = Variable(x_tensor, requires_grad=True)
y = Variable(y_tensor, requires_grad=True)
z = x * y + 10
p = z.sum()
p.backward()
print('x:', x.data)
print('y:', y.data)
print('dp / dx:', x.grad.data)
print('dp / dy:', y.grad.data)

In [ ]:
X = Variable(torch.randn(4).view(1, 4), requires_grad=True)
w = Variable(torch.randn(4).view(4, 1), requires_grad=True)
b = 7
T = torch.mm(X, w) + b
Z = torch.sigmoid(T)

In [ ]:
Z.backward()

In [ ]:
T.grad

In [ ]:
T.grad.detach_()

In [ ]:
T.grad.zero_(), w.grad.zero_(), X.grad.zero_()

### Обнуление градиентов

In [ ]:
x_tensor = torch.Tensor([1, 1, 1, 1])
x = Variable(x_tensor, requires_grad=True)
y = x ** 2
p = y.sum()
p.backward()
print('x:', x.data.numpy())
print('dp / dx:', x.grad.data.numpy())
x.data -= 0.5
y = 1 / x
p = y.sum()
p.backward()
print('x:', x.data.numpy())
print('dp / dx:', x.grad.data.numpy())

In [ ]:
x_tensor = torch.Tensor([1, 1, 1, 1])
x = Variable(x_tensor, requires_grad=True)
y = x ** 2
p = y.sum()
p.backward()
print('x:', x.data.numpy())
print('dp / dx:', x.grad.data.numpy())
x.grad.detach_()       # extracting gradient Variable from the previous computational graph (optional)
x.grad.data.zero_()    # zero gradinents
x.data -= 0.5
y = 1 / x
p = y.sum()
p.backward()
print('x:', x.data.numpy())
print('dp / dx:', x.grad.data.numpy())

## Leaf vs Non-leaf Variable

Градиенты будут сохранены и доступны для использования только для `leaf-variable`.
Такое поведение по умолчанию сделано ради экономии памяти.

In [ ]:
x = Variable(torch.randn(4), requires_grad=True)  # leaf variable
y = x + 1                                         # not a leaf variable
p = y.sum()                                       # not a leaf variable
p.backward()
print('x.data:', x.data.numpy())
print('y.data:', y.data.numpy())
print('p.data:', p.data.numpy())
print('x.grad:', x.grad.data.numpy())
print('y.grad:', y.grad)
print('p.grad:', p.grad)

In [ ]:
x = Variable(torch.randn(4), requires_grad=True)  # leaf variable
y = Variable(torch.randn(4), requires_grad=True)  # leaf variable
z = x + y    # not a leaf variable
p = z.sum()  # not a leaf variable
p.backward()
print('x.grad:', x.grad.data.numpy())
print('y.grad:', y.grad.data.numpy())
print('z.grad:', z.grad)
print('p.grad:', p.grad)
print('x.is_leaf:', x.is_leaf)
print('y.is_leaf:', y.is_leaf)
print('z.is_leaf:', z.is_leaf)
print('p.is_leaf:', p.is_leaf)

## Листовые вершины без градиентов
Листовые вершины, в которых не требуется вычислять градиент, создаются с помощью `Variable(..., requires_grad=False)`.
Для корректного вызова `.backward()` требуется, чтобы хотя бы для одной листовой вершины требовался градиент.

In [ ]:
x = Variable(torch.randn(4), requires_grad=True)   # leaf variable
y = Variable(torch.randn(4), requires_grad=False)  # leaf variable
z = x + y    # not a leaf variable
p = z.sum()  # not a leaf variable
p.backward()
print('x.grad:', x.grad.data.numpy())
print('y.grad:', y.grad)
print('z.grad:', z.grad)
print('p.grad:', p.grad)
print('x.is_leaf:', x.is_leaf)
print('y.is_leaf:', y.is_leaf)
print('z.is_leaf:', z.is_leaf)
print('p.is_leaf:', p.is_leaf)

Обратите внимание, что для вычисления градиента нужно, чтобы хотя бы одна листовая вершина графа вычисления функции
имела `requires_grad=True`.

In [ ]:
# will not work:
x = Variable(torch.randn(4), requires_grad=False)  # leaf variable
y = Variable(torch.randn(4), requires_grad=False)  # leaf variable
z = x + y    # not a leaf variable
p = z.sum()  # not a leaf variable
p.backward()

## Градиенты промежуточных вершин
Для промежуточных вершин мы можем запросить сохранение градиентов с помощью функции `.retain_grad()`.

In [ ]:
x = Variable(torch.randn(4), requires_grad=True)   # leaf variable
z = Variable(torch.randn(4), requires_grad=True)   # leaf variable
w = z * 2      # not a leaf variable
y = x * w + 1  # forward pass before retaining gradient is ok
p = y.sum()

w.retain_grad()

p.backward()
print('dp / dx:', x.grad.data.numpy())
print('dp / dw:', w.grad.data.numpy())
print('dp / dz:', z.grad.data.numpy())

Обратим внимание, что даже при наличии в графе вычислений не-листовых вершин, требующих вычисления градиентов,
`.backward()` выдает ошибку.

In [ ]:
# will not work
x = Variable(torch.randn(4), requires_grad=False)   # leaf variable
z = Variable(torch.randn(4), requires_grad=False)   # leaf variable
w = z * 2      # not a leaf variable
y = x * w + 1  # forward pass before retaining gradient is ok
p = y.sum()

w.retain_grad()

p.backward()

## Отстреливаем себе ноги (НЕ НАДО так делать)

Конвертировать Variable в Tensor и обратно:
backward pass не проходит через Tensor, даже если он был сконвертирован из другого Variable.

In [ ]:
# x out of the computational graph
x = torch.autograd.Variable(torch.Tensor([1, 2, 3, 4]), requires_grad=True)
y = torch.autograd.Variable(x.data * 2, requires_grad=True)   # the bad conversion is here
z = 3 * y + 1
p = z.mean()
p.backward()
print('dp / dx:', x.grad)
print('dp / dy:', y.grad.data.numpy())

Менять размерность тензоров в Variable, но не обнулять градиенты (`.grad.zero_()` сохраняет размер, `.grad = None` не сохраняет).

In [ ]:
x = torch.autograd.Variable(torch.Tensor([1, 2, 3, 4]), requires_grad=True)
y = torch.autograd.Variable(torch.Tensor([1, 2, 3, 4]), requires_grad=True)
z = x * y + 1
z.sum().backward()
print('dz / dx:', x.grad.data.numpy())
print('dz / dy:', y.grad.data.numpy())

x.grad.zero_()
z.grad.zero_()
#x.grad = None
#z.grad = None

x.data = torch.Tensor([1, 2, 3])
y.data = torch.Tensor([1, 2, 3])
z = x * y + 1
z.sum().backward()
print('dz / dx:', x.grad.data.numpy())
print('dz / dy:', y.grad.data.numpy())

Менять значения Variable после вычисления каких-то других выражений с ним и рассчитывать,
что градиент от тех выражений будет учитывать новое значение.

In [ ]:
x = torch.autograd.Variable(torch.Tensor([1, 2, 3, 4]), requires_grad=True)
y = torch.autograd.Variable(torch.Tensor([1, 2, 3, 4]), requires_grad=True)
z = y ** 2

z.data = torch.Tensor([1, 2, 3, 4])  # changing .data before computation matters
p = x * z
x.data = torch.Tensor([1, 1, 1])     # changing .data after computation doesn't affect gradients

p.sum().backward()
print('d p_sum / dx:', x.grad.data.numpy())
print('d p_sum / dy:', y.grad.data.numpy())

Тысячи способов прострелить себе ногу, если использовать механизм автоматического дифференцирования
любым другим нетрадиционным образом.

## Пример задачи

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
X = torch.randn(50, 10)
b = torch.randn(2)
W = torch.randn(10, 2)

Наркоманская функция потерь, вынуждающая линейное преобразование переводить точки из многомерного пространства в двумерное на единичную окружность. Для оптимизации использовать градиентный спуск по параметрам преобразования.

Линейное преобразование точки $x$ из десятимерного пространства в точку $y$ двумерного пространства с весами преобразования $W$ и $b$:
$$y = Wx + b$$

Норма в двумерном пространстве – евклидова:
$$||y||_2 = \sqrt{y_1^2 + y_2^2}$$

Функция потерь $f_0$ штрафует расстояние от получившейся точки $y$ до единичной окружности:
$$f_0(x, W, b) = 0.5 \cdot \big| ||y||_2 - 1 \big| + \big( ||y||_2 - 1 \big)^2$$

К сожалению, оптимизация функции $f_0$ по $W$ и $b$ может быть проведена аналитически
и приводит к тривиальному решению $W = 0$, $b = (1, 0)$.
Чтобы избежать такого решения, вводим штраф на близость получившейся точки к вектору $b$, который обращается в 0, если расстояние до вектора $b$ более 1:
$$f_1(x, W, b) = \max\big(0, \frac{1}{||y - b||_2} - 1\big)$$

Итоговая функция потерь:
$$f(x, W, b) = f_0(x, W, b) + f_1(x, W, b)$$

Нужно решить следующую оптимизационную задачу:
$$\frac{1}{N}\sum\limits_{i = 1}^N f(x_i, W, b) \to \min\limits_{W, b}$$

In [17]:
def f(X, W, b):
    # your code here
    Y = torch.matmul(X, W) + b
    y_norm = torch.sqrt((Y**2).sum(dim=1))
    f_0 = 0.5*torch.abs(y_norm-1) + (y_norm-1)**2
    y_b_norm = torch.sqrt(torch.sum((Y - b)**2, dim=1))
    f_1 = torch.max(Variable(torch.zeros(50, 1)), 1/y_b_norm - 1)
    L = f_0 + f_1 
    Q = L.sum()/(L.shape[0]*L.shape[1])
    return Q

In [18]:
from torch.optim import Adam

In [19]:
variable_X = Variable(X)
variable_W = Variable(W, requires_grad=True)
variable_b = Variable(b, requires_grad=True)

NameError: name 'W' is not defined

In [20]:
print(f(variable_X, variable_W, variable_b).data)

NameError: name 'variable_W' is not defined

In [21]:
opt = Adam([variable_W, variable_b], lr=0.003)

NameError: name 'variable_W' is not defined

In [22]:
num_epoch = 10000
for i in range(num_epoch):
    opt.zero_grad()
    loss = f(variable_X, variable_W, variable_b)
    loss.backward()
    opt.step()

NameError: name 'opt' is not defined

In [23]:
print(loss)

NameError: name 'loss' is not defined

In [24]:
plt.figure(figsize=(5, 5))
Y = X.mm(W).add(b)
plt.scatter(Y[:, 0], Y[:, 1])

NameError: name 'plt' is not defined

In [25]:
plt.figure(figsize=(5, 5))
Y = variable_X.mm(variable_W).add(variable_b)
plt.scatter(Y[:, 0], Y[:, 1])

NameError: name 'plt' is not defined

# Время писать нейросеть

## Загружаем данные

In [26]:
from sklearn.datasets import load_digits

X, y = load_digits(return_X_y=True)
X.shape, y.shape

((1797, 64), (1797,))

In [27]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)
X_train.shape, X_test.shape

((1347, 64), (450, 64))

In [28]:
X_train = torch.from_numpy(X_train).type(torch.FloatTensor)
y_train = torch.from_numpy(y_train).type(torch.LongTensor)
X_test = torch.from_numpy(X_test).type(torch.FloatTensor)
y_test = torch.from_numpy(y_test).type(torch.LongTensor)

Подсказка: нейросети крайне плохо обучаются, если подаваемые им на вход значения велики по модулю.
Поэтому перед обучением нейросети каждый признак независимо нормируют
(исключение – сверточные нейросети, там нормируют изображение поканально, а не попиксельно, но об этом потом).

Можно использовать разные нормировки.
Наиболее популярно вычитать среднее и делить на дисперсию (нужно внимательно подходить к этому методу,
когда выборочная дисперсия мала или равна нулю, и обрабатывать такие случаи отдельно).
Можно также вычитать медиану и делить на интерквартильный размах, масштабировать все данные в отрезок $[-1, 1]$, etc

Нужно реализовать свою нормировку данных здесь:

In [29]:
# your code here
mean = X_train.mean(dim=0)
std = X_train.std(dim=0)
std.apply_(lambda x: x if x>0 else 0.0001)
X_train = (X_train - mean)/std
X_test = (X_test - mean)/std

## Определяем слои нейросети

In [30]:
class Module:
    def __init__(self):
        self.training = True
        self.children = []

    def forward(self, *input):
        raise NotImplementedError

    def parameters(self):
        """Returns list of parameters of module and its children."""
        res = []
        for submodule in self.children:
            res += submodule.parameters()
        for param in res:
            if not isinstance(param, Variable):
                raise Exception('Parameters must be Variables.')
        return res
    
    def zero_grad(self):
        """Sets gradients of all model parameters to zero."""
        for p in self.parameters():
            if p.grad is not None:
                p.grad.detach_()   # detachs gradient Variable from the computational graph
                p.grad.zero_()

    def train(self):
        """Sets module into train mode (for DropOut, BatchNorm, etc)."""
        self.training = True
        for submodule in self.children:
            submodule.train()

    def eval(self):
        """Sets module into evaluation mode."""
        self.training = False
        for submodule in self.children:
            submodule.eval()

In [31]:
class Dense(Module):
    def __init__(self, input_units, output_units):
        """A dense layer is a layer which performs a learned affine transformation:
        f(x) = W x + b
        """
        super(Dense, self).__init__()
        # initialize weights with small random numbers from normal distribution
        self.weights = Variable(torch.randn(input_units, output_units)*0.00001, requires_grad=True)  # your code here
        self.biases = Variable(torch.randn(1, output_units)*0.00001, requires_grad=True)   # your code here

    def parameters(self):
        return [self.weights, self.biases]
        
    def forward(self, input):
        """Performs an affine transformation:
        f(x) = W x + b
        input shape:  [batch, input_units]  (Variable)
        output shape: [batch, output units] (Variable)
        """
        # your code here
        self.output = torch.matmul(input, self.weights) + self.biases
        
        return self.output

In [32]:
class ReLU(Module):
    def __init__(self):
        """ReLU layer simply applies elementwise rectified linear unit to all inputs."""
        super(ReLU, self).__init__()

    def parameters(self):
        return []  # ReLU has no parameters
    
    def forward(self, input):
        """Applies elementwise ReLU to [batch, num_units] Variable matrix."""
        # your code here
        self.output = torch.nn.functional.relu(input)
        
        return self.output

In [33]:
class LogSoftmax(Module):
    def __init__(self):
        super(LogSoftmax, self).__init__()

    def parameters(self):
        return []
        
    def forward(self, input):
        """Applies softmax to each row and then applies component-wise log.
        Input shape:  [batch, num_units] (Variable)
        Output shape: [batch, num_units] (Variable)
        """
        # your code here
        self.output = torch.nn.functional.log_softmax(input, dim=1)
        
        return self.output

In [34]:
class MyNetwork(Module):
    def __init__(self, input_size, hidden_layers_size, hidden_layers_number, output_size):
        super(MyNetwork, self).__init__()

        network = []
        network.append(Dense(input_size, hidden_layers_size))
        network.append(ReLU())
        for i in range(hidden_layers_number - 1):
            network.append(Dense(hidden_layers_size, hidden_layers_size))
            network.append(ReLU())
        network.append(Dense(hidden_layers_size, output_size))
        network.append(LogSoftmax())

        self.children = network

    def forward(self, input):
        """Applies all layers of neural network to the input.
        Input shape:  [batch, num_units] (Variable)
        Output shape: [batch, num_units] (Variable)
        """
        # your code here
        output = input
        for layer in self.children:
            output = layer.forward(output)
        
        return output

In [35]:
network = MyNetwork(X_train.shape[1], 32, 1, 10)

## Определяем функцию потерь

In [36]:
def crossentropy(activations, target):
    """Returns negative log-likelihood of target under model
    represented by activations (log probabilities of classes).
    Activations shape: [batch, num_classes] (Variable)
    Target shape:      [batch]              (Variable)
    Output shape: 1 (scalar, Variable)
    """
    # your code here
    #loss = - torch.sum(activations[:, target])
    loss = - torch.sum(activations[np.arange(len(target)), target])
    return loss

## Оптимизатор SGD

In [37]:
class SGDOptimizer:
    def __init__(self, parameters, learning_rate):
        self.parameters = parameters
        self.learning_rate = learning_rate

    def step(self):
        """Make one optimization step for parameters in-place.
        Assumes that all parameters are Variable with computed gradient.
        """
        for param in self.parameters:
            param.data -= self.learning_rate * param.grad.data

In [38]:
import torch.utils.data

In [39]:
def run_epoch(dataset, network, prefix='Test loss:', optimizer=None):
    # Change mode for all layers.
    if optimizer:
        network.train()
    else:
        network.eval()

    batch_size = 100
    batchgenerator = torch.utils.data.DataLoader(dataset, batch_size, True)

    avg_loss = 0
    for i, (batch_data, batch_target) in enumerate(batchgenerator):
        batch_output = network.forward(Variable(batch_data))
        batch_loss = crossentropy(batch_output, Variable(batch_target))
        #print(batch_output)
        batch_loss.backward()
        batch_loss = batch_loss.data.numpy()[0]
        avg_loss += (batch_loss - avg_loss) / (i + 1)
        if optimizer:
            optimizer.step()
            network.zero_grad()
    print(prefix, avg_loss, flush=True)

In [40]:
train_dataset = torch.utils.data.TensorDataset(X_train, y_train)
test_dataset = torch.utils.data.TensorDataset(X_test, y_test)

network = MyNetwork(X_train.shape[1], 32, 1, 10)
sgd = SGDOptimizer(network.parameters(), 0.005)

num_epochs = 20
for i in range(num_epochs):
    run_epoch(train_dataset, network, 'Train loss:', sgd)
    run_epoch(test_dataset, network, 'Test loss:',  None)

Train loss: 221.693268367
Test loss: 207.356959534
Train loss: 221.709107535
Test loss: 207.325334167
Train loss: 215.913970947
Test loss: 184.784657288
Train loss: 128.114058086
Test loss: 78.4177284241
Train loss: 45.9722008024
Test loss: 32.0823406219
Train loss: 24.3204667909
Test loss: 18.1917114258
Train loss: 15.4498525006
Test loss: 11.9230111122
Train loss: 11.4012275764
Test loss: 9.33497753143
Train loss: 9.12822192056
Test loss: 7.59143161774
Train loss: 7.5957518816
Test loss: 6.53410797119
Train loss: 6.67689425605
Test loss: 5.52004499435
Train loss: 5.76055191244
Test loss: 4.96747469902
Train loss: 5.12720776456
Test loss: 4.31735992432
Train loss: 4.49048909119
Test loss: 3.93370199203
Train loss: 4.01060628039
Test loss: 3.51529927254
Train loss: 3.54166832141
Test loss: 3.1078253746
Train loss: 3.22932979039
Test loss: 2.78191628456
Train loss: 2.89354721563
Test loss: 2.54150979519
Train loss: 2.56870407292
Test loss: 2.28947069645
Train loss: 2.42793481691
Test lo

## Больше оптимизаторов Б-гу Оптимизации!

In [61]:
class SGDMomentumOptimizer:
    def __init__(self, parameters, learning_rate=0.01, momentum=0.9):
        self.parameters = parameters
        self.learning_rate = learning_rate
        self.momentum = momentum
        # your code here
        self.lag_parameters = self.parameters
        

    def step(self):
        """Make one optimization step for parameters in-place.
        Assumes that all parameters are Variable with computed gradient.
        """
        # your code here
        previous = self.parameters 
        for param, lag_param in zip(self.parameters, self.lag_parameters):
            param.data -= self.learning_rate * param.grad.data + self.momentum*(param.data-lag_param.data)
            self.lag_parameters = previous

In [62]:
class RMSPropOptimizer:
    def __init__(self, parameters, learning_rate=0.01, beta=0.9, eps=1e-8):
        self.parameters = parameters
        self.learning_rate = learning_rate
        self.beta = beta
        self.eps = eps
        # your code here
        self.v = [Variable(torch.zeros(tensor.size()), requires_grad=True) for tensor in parameters]
        
    def step(self):
        """Make one optimization step for parameters in-place.
        Assumes that all parameters are Variable with computed gradient.
        """
        # your code here
        for param, v in zip(self.parameters, self.v):
            v.data = self.beta*v.data + (1-self.beta)*(param.grad.data)**2 
            param.data -= self.learning_rate*param.grad.data/torch.sqrt(v.data + self.eps)

In [63]:
class AdamOptimizer:
    def __init__(self, parameters, learning_rate=0.01, beta1=0.9, beta2=0.999, eps=1e-8):
        self.parameters = parameters
        self.learning_rate = learning_rate
        self.beta1 = beta1
        self.beta2 = beta2
        self.eps = eps
        # your code here
        self.m = [Variable(torch.zeros(tensor.size())) for tensor in parameters]
        self.v = [Variable(torch.zeros(tensor.size())) for tensor in parameters]
        self.t = 0
        

    def step(self):
        """Make one optimization step for parameters in-place.
        Assumes that all parameters are Variable with computed gradient.
        """
        # your code here
        self.t += 1
        for param, m, v in zip(self.parameters, self.m, self.v):
            m.data = self.beta1 * m.data + (1 - self.beta1) * param.grad.data
            v.data = self.beta2 * v.data + (1 - self.beta2) * param.grad.data**2
            m_hat = m.data / (1 - self.beta1**self.t)
            v_hat = v.data / (1 - self.beta2**self.t)
            param.data -= (self.learning_rate / (torch.sqrt(v_hat) + self.eps)) * m_hat

In [64]:
train_dataset = torch.utils.data.TensorDataset(X_train, y_train)
test_dataset = torch.utils.data.TensorDataset(X_test, y_test)

In [65]:
network = MyNetwork(X_train.shape[1], 32, 1, 10)
optim = SGDMomentumOptimizer(network.parameters())

num_epochs = 20
for i in range(num_epochs):
    run_epoch(train_dataset, network, 'Train loss:', optim)
    run_epoch(test_dataset, network, 'Test loss:',  None)

Train loss: 221.792132786
Test loss: 207.497396851
Train loss: 208.627980913
Test loss: 142.205712891
Train loss: 63.789528438
Test loss: 28.9509429932
Train loss: 18.8545825141
Test loss: 14.9617086411
Train loss: 11.3087986708
Test loss: 9.23672075272
Train loss: 8.88867645604
Test loss: 8.56956825256
Train loss: 10.5502818993
Test loss: 11.6600829124
Train loss: 5.79795605796
Test loss: 4.49802207947
Train loss: 3.85666584117
Test loss: 3.96704499722
Train loss: 3.26321063723
Test loss: 4.8798538208
Train loss: 2.9778298395
Test loss: 2.25139176846
Train loss: 2.11880375658
Test loss: 1.9001418829
Train loss: 1.66512248346
Test loss: 1.57926323414
Train loss: 1.44689011148
Test loss: 1.42686934471
Train loss: 1.26057777234
Test loss: 1.22988757491
Train loss: 1.1117405636
Test loss: 1.08971599936
Train loss: 0.988719438868
Test loss: 0.948154628277
Train loss: 0.89676123006
Test loss: 0.906381630898
Train loss: 0.829638031977
Test loss: 0.781771972775
Train loss: 0.752192045961
Test

In [66]:
network = MyNetwork(X_train.shape[1], 32, 1, 10)
optim = RMSPropOptimizer(network.parameters())

num_epochs = 20
for i in range(num_epochs):
    run_epoch(train_dataset, network, 'Train loss:', optim)
    run_epoch(test_dataset, network, 'Test loss:',  None)

Train loss: 124.717394829
Test loss: 44.1945404053
Train loss: 27.9040530069
Test loss: 19.4143827438
Train loss: 14.9262724604
Test loss: 12.0128164291
Train loss: 9.42871136325
Test loss: 7.67577753067
Train loss: 5.96195571763
Test loss: 5.70320343971
Train loss: 4.07281899452
Test loss: 4.07001588345
Train loss: 2.82239182506
Test loss: 3.31514434814
Train loss: 1.86271859067
Test loss: 2.17463693023
Train loss: 1.27579508881
Test loss: 1.67005878687
Train loss: 0.902630478144
Test loss: 1.4451825738
Train loss: 0.682061655181
Test loss: 0.992381381989
Train loss: 0.336626931493
Test loss: 0.818373528123
Train loss: 0.235988681072
Test loss: 0.711748760939
Train loss: 0.211213894721
Test loss: 0.634506493807
Train loss: 0.294234647815
Test loss: 0.171560300887
Train loss: 0.0915165645336
Test loss: 0.094993891567
Train loss: 0.0591958463857
Test loss: 0.0604913912714
Train loss: 0.0530295801083
Test loss: 0.0739043958485
Train loss: 0.031776202643
Test loss: 0.039821170643
Train lo

In [67]:
network = MyNetwork(X_train.shape[1], 32, 1, 10)
optim = AdamOptimizer(network.parameters())

num_epochs = 20
for i in range(num_epochs):
    run_epoch(train_dataset, network, 'Train loss:', optim)
    run_epoch(test_dataset, network, 'Test loss:',  None)

Train loss: 180.333901269
Test loss: 96.9577812195
Train loss: 65.6407427107
Test loss: 42.5596565247
Train loss: 30.3924466542
Test loss: 21.1837905884
Train loss: 15.3301186562
Test loss: 11.2946683884
Train loss: 9.17791019167
Test loss: 7.36945893764
Train loss: 6.34908700841
Test loss: 5.56227004528
Train loss: 4.66237752778
Test loss: 4.34610834122
Train loss: 3.4843770521
Test loss: 3.63975114822
Train loss: 2.66346748812
Test loss: 3.06203801632
Train loss: 2.10765596798
Test loss: 2.50851314068
Train loss: 1.68493120585
Test loss: 2.13663833141
Train loss: 1.41378126826
Test loss: 1.86096903086
Train loss: 1.15184976799
Test loss: 1.59302366972
Train loss: 0.987093228315
Test loss: 1.44169704914
Train loss: 0.851644066828
Test loss: 1.28307608962
Train loss: 0.731254270034
Test loss: 1.20932716131
Train loss: 0.62611842049
Test loss: 1.0868888855
Train loss: 0.542785914881
Test loss: 1.01632710099
Train loss: 0.487751379609
Test loss: 0.92935885191
Train loss: 0.435861863196
T

## Эксперименты с DropOut

Этот пункт обязателен к выполнению.
Для того, чтобы получить бонусный балл за этот пункт, нужно эффективно реализовать DropOut:
не вычислять активации выкинутых нейронов, прежде чем их обнулить.

In [68]:
class DenseWithDropOut(Module):
    def __init__(self, input_units, output_units, dropout_rate, nonlinearity):
        """A dense layer is a layer which performs a learned
        affine transformation and applies dropout:
        m ~ Bernoulli(1 - p, size=output_units)
        f(x) = g(W x + b) o m
        """
        super(DenseWithDropOut, self).__init__()
        self.dropout_rate = dropout_rate
        self.nonlinearity = nonlinearity
        # initialize weights with small random numbers from normal distribution
        self.weights = Variable(torch.randn(input_units, output_units)*0.0001, requires_grad=True) # your code here
        self.biases = Variable(torch.randn(1, output_units)*0.0001, requires_grad=True) # your code here
        self.output_units = output_units

    def parameters(self):
        return [self.weights, self.biases]
        
    def forward(self, input, idxs_x):
        """Performs an affine transformation with dropout.
        In training mode:
        m ~ Bernoulli(1 - p, size=output_units)
        f(x) = g(W x + b) o m
        In evaluation mode:
        f(x) = g(W x + b) (1 - p)
        input shape:  [batch, input_units]  (Variable)
        output shape: [batch, output units] (Variable)
        """
        # your code here
        '''
        Не работает быстрее
        if self.training:
            y = np.ones(self.output_units)
            num_zeroes = np.random.binomial(self.output_units, self.dropout_rate)
            idxs = np.random.choice(self.output_units, num_zeroes, replace=False)
            y[idxs] = 0
            y = y.reshape((1, self.output_units))
            y = Variable(torch.from_numpy(y))
            output = self.nonlinearity(torch.matmul(input, self.weights) + self.biases)
            output = output.float()*y.float() 
        '''
        if self.training:
            """В данном форварде дропаут реализован так, что на вход в следующий слой помимо output
               передаются также индексы весов, на которые нам надо домножать инпут. Скорость работы
               получается больше, чем если восстанавливать нули во всей матрице после дропаута.
            """
            y = np.ones(self.output_units)
            num_zeroes = np.random.binomial(self.output_units, 1-self.dropout_rate)
            idxs = np.random.choice(self.output_units, num_zeroes, replace=False)
            if num_zeroes is 0:
                idxs = np.array([0])
            idxs = torch.from_numpy(idxs)
            idxs = Variable(idxs).long()
            weights_remain = self.weights.index_select(1, idxs)
            biases_remain = self.biases.index_select(1, idxs)
            weights_remain = weights_remain.index_select(0, idxs_x.long())
            print('Input shape is: ', input.shape, 'Weights shape is: ', weights_remain.shape)
            output = [self.nonlinearity(torch.matmul(input, weights_remain) + biases_remain), idxs]
        else:
            output = self.nonlinearity(torch.matmul(input, self.weights) + self.biases)*(1-self.dropout_rate)
        
        return output

Проверяем, верно ли, что полносвязная сеть с dropout работает быстрее, чем обычная полносвязная сеть, поскольку на каждом проходе вычисляются произведения матриц меньшего размера.

In [71]:
width = 2000
network1 = [
    DenseWithDropOut(width, width, 0.9, lambda x: ReLU().forward(x)),
    DenseWithDropOut(width, width, 0.9, lambda x: ReLU().forward(x)),
    DenseWithDropOut(width, width, 0.9, lambda x: ReLU().forward(x)),
    DenseWithDropOut(width, 1, 0, lambda x: x)
]
network2 = [
    Dense(width, width),
    ReLU(),
    Dense(width, width),
    ReLU(),
    Dense(width, width),
    ReLU(),
    Dense(width, 1)
]
X = torch.randn(10000, width)

# check whether DenseWithDropOut works faster than Dense
def test_network(network):
    x = Variable(X)
    idxs_x = Variable(torch.from_numpy(np.arange(0, x.shape[1], 1))).long()
    for layer in network:
        if isinstance(x, list):
            x = x[0]
            idxs_x = x[1]   
        x = layer.forward(x, idxs_x)
    if isinstance(x, list):
        x[0].mean().backward()
    else:
        x.mean().backward()

    for layer in network:
        x = layer.zero_grad()

def test_network_2(network):
    x = Variable(X)
    for layer in network:
        x = layer.forward(x)
    x.mean().backward()
    for layer in network:
        x = layer.zero_grad()

%timeit test_network(network1)
%timeit test_network_2(network2)

Input shape is:  torch.Size([10000, 2000]) Weights shape is:  torch.Size([2000, 203])
Input shape is:  torch.Size([10000, 203]) Weights shape is:  torch.Size([203, 232])
Input shape is:  torch.Size([10000, 232]) Weights shape is:  torch.Size([232, 195])
Input shape is:  torch.Size([10000, 195]) Weights shape is:  torch.Size([195, 1])
Input shape is:  torch.Size([10000, 2000]) Weights shape is:  torch.Size([2000, 206])
Input shape is:  torch.Size([10000, 206]) Weights shape is:  torch.Size([206, 205])
Input shape is:  torch.Size([10000, 205]) Weights shape is:  torch.Size([205, 209])
Input shape is:  torch.Size([10000, 209]) Weights shape is:  torch.Size([209, 1])
Input shape is:  torch.Size([10000, 2000]) Weights shape is:  torch.Size([2000, 225])
Input shape is:  torch.Size([10000, 225]) Weights shape is:  torch.Size([225, 193])
Input shape is:  torch.Size([10000, 193]) Weights shape is:  torch.Size([193, 208])
Input shape is:  torch.Size([10000, 208]) Weights shape is:  torch.Size([2

Для более узких слоев, меньших dropout rate и меньших размеров батча увеличение производительности не настолько существенно или может вообще отсутствовать.